In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
driver = webdriver.Chrome()

In [3]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [4]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[4]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 두산 베어스 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[5]').click()

# 수비 > 내야 선택
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[6]/ul/li[4]/a').click()

In [5]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [6]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [7]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'GS', 'IP', 'BAFZ', 'RF9', 'ofAss', 'F12', 'F12', 'F12', 'F23', 'F23', 'F23', 'F34', 'F34', 'F34', 'H13', 'H13', 'H13', 'H24', 'H24', 'H24', 'D14', 'D14', 'D14', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도']
1 ['Rank', 'Name', 'Team', '수비 관련 득점 기여도-종합 RAA', 'G', 'GS', 'IP', 'BAFZ', 'RF9', 'ofAss', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'A', 'N', '%', 'Range RAA', 'Arm RAA', 'Err RAA', '종합 RAA▼', '/144']
2 ['1', '조수행', '24  LF', '5.74', '26', '20', '185.0', '35.5', '2.68', '1', '0', '9', '0.0', '0', '2', '0.0', '0', '0', '', '1', '13', '', '0', '1', '', '3', '6', '', '5.85', '-0.30', '0.18', '5.74', '22.94']
3 ['2', '김대한', '24  RF', '2.28', '16', '10', '97.0', '52.6', '2.41', '0', '0', '5', '0.0', '0', '4', '0.0', '0', '1', '0.0', '3', '4', '0.0', '0', '3', '0.0', '0', '1', '0.0', '2.85', '-0.42', '-0.15', '2.28', '12.60']
4 ['3', '정수빈', '24  CF', '0.26', '42', '42', '347.0', '

In [8]:
num = (len(p) // 10)-1
start_num = num
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [15]:
data=pd.DataFrame(p)
data=data[2:]
data.columns = ["Rank", "Name", "Position","Total RAA2","G", "GS","IP", "BAFZ","RF9","ofAss","F12 A","F12 N","F12 %","F23 A","F23 N","F23 %","F34 A","F34 N","F34 %","H13 A", "H13 N", "H13 %", "H24 A", "H24 N", "H24 %", "D14 A", "D14 N", "D14 %", "Range RAA", "Arm RAA", "Err RAA","Total RAA", "/144"]
data.drop(columns=['Total RAA2'], inplace=True)
data['Position'].replace({"24  ":""}, regex=True, inplace=True)
data['Team']="Doosan"
print(data)

   Rank Name Position   G  GS     IP   BAFZ   RF9 ofAss F12 A  ... H24 %  \
2     1  조수행       LF  26  20  185.0   35.5  2.68     1     0  ...         
3     2  김대한       RF  16  10   97.0   52.6  2.41     0     0  ...   0.0   
4     3  정수빈       CF  42  42  347.0   45.2  2.59     2     1  ...   0.0   
5     4  조수행       RF   7   2   22.0   40.0  2.05     1     0  ...         
6     5  전다민       CF   1   0    2.0         0.00     0     0  ...         
7     6  김대한       CF   1   0    2.0         0.00     0     0  ...         
8     7  김태근       RF   5   0    9.0    0.0  3.00     0     0  ...         
9     8  홍성호       RF   2   0    4.0         0.00     0     0  ...         
10    9  라모스       RF  30  30  242.0   30.6  1.64     0     0  ...         
11   10  조수행       CF   4   0   12.0    0.0  2.25     1     0  ...         
12   11  김인태       RF   1   1    6.0  100.0  3.00     0     0  ...         
13   12  김태근       LF   5   3   30.2   28.6  2.35     0     0  ...         
14   13  김인태

In [16]:
data.to_csv("Doosan_field_OF.csv", index=False)